In [ ]:
import yaml
import os
import pathlib
from pathlib import Path

import scanpy as sc
import torch
import numpy as np
import pandas as pd

from scSLAT.model import scanpy_workflow,spatial_match
from scSLAT.metrics import global_score
from scSLAT.viz import match_3D_multi

In [ ]:
sc.set_figure_params(dpi_save=200,dpi=150)

In [ ]:
# parameter cells
adata1_file = ''
adata2_file = ''
method = ''
metric_file = ''
emb0_file = ''
emb1_file = ''

In [ ]:
adata1 = sc.read_h5ad(adata1_file)
adata2 = sc.read_h5ad(adata2_file)
solt = 'X_' + method.lower()

In [ ]:
embd0 = adata1.obsm[solt]
embd1 = adata2.obsm[solt]
best, index, distance = spatial_match([embd0, embd1],smooth=False)
matching = np.array([range(index.shape[0]), best])

In [ ]:
if 'visium' and 'DLPFC' in adata1_file:
    biology_meta = 'cell_type'
    topology_meta = 'layer_guess'
elif 'merfish' and 'hypothalamic' in adata1_file:
    biology_meta = 'Cell_class'
    topology_meta = 'region'
elif 'stereo' and 'embryo' in adata1_file:
    biology_meta = 'annotation'
    topology_meta = 'region'

In [ ]:
overall_score = global_score([adata1,adata2], matching.T, biology_meta, topology_meta)
celltype_score = global_score([adata1,adata2], matching.T, biology_meta=biology_meta)
region_score = global_score([adata1,adata2], matching.T, topology_meta=topology_meta)

In [ ]:
time_key =  method.lower() + '_time'
run_time = adata1.uns[time_key]

# Save

In [ ]:
metric_dic = {}
metric_dic['global_score'] = overall_score
metric_dic['celltype_score'] = celltype_score
metric_dic['region_score'] = region_score
metric_dic['run_time'] = run_time

with open(metric_file, "w") as f:
    yaml.dump(metric_dic, f)

In [ ]:
np.savetxt(emb0_file, adata1.obsm[solt], delimiter=',')
np.savetxt(emb1_file, adata2.obsm[solt], delimiter=',')

# Plot

In [ ]:
adata_all = adata1.concatenate(adata2)
out_dir = Path(os.path.dirname(metric_file))
sc.pp.neighbors(adata_all, metric="cosine", use_rep=solt)
sc.tl.umap(adata_all)
sc.pl.umap(adata_all, color=biology_meta,save=out_dir / 'biology.pdf')
sc.pl.umap(adata_all, color=topology_meta,save=out_dir / 'topology.pdf')
sc.pl.umap(adata_all, color="batch",save=out_dir / 'batch.pdf')

In [ ]:
adata1_df = pd.DataFrame({'index':range(embd0.shape[0]),
                          'x': adata1.obsm['spatial'][:,0],
                          'y': adata1.obsm['spatial'][:,1],
                          'celltype':adata1.obs[biology_meta],
                          'region':adata1.obs[topology_meta]})
adata2_df = pd.DataFrame({'index':range(embd1.shape[0]),
                          'x': adata2.obsm['spatial'][:,0],
                          'y': adata2.obsm['spatial'][:,1],
                          'celltype':adata2.obs[biology_meta],
                          'region':adata2.obs[topology_meta]})
matching = np.array([range(index.shape[0]), best])

In [ ]:
multi_align = match_3D_multi(adata1_df, adata2_df,matching,meta='celltype',
                             scale_coordinate=True,subsample_size=300,exchange_xy=False)

multi_align.draw_3D(size= [7, 8], line_width =1, point_size=[0.8,0.8], hide_axis=True, show_error=False, save=out_dir / 'match_by_celltype.pdf')

In [ ]:
multi_align = match_3D_multi(adata1_df, adata2_df,matching,meta='region',
                             scale_coordinate=True,subsample_size=300,exchange_xy=False)

multi_align.draw_3D(size= [7, 8], line_width =1, point_size=[0.8,0.8], hide_axis=True, show_error=False, save=out_dir / 'match_by_region.pdf')